In [165]:
import argparse
import sys
import os
from collections import defaultdict
from pprint import pprint

from tqdm import tqdm

sys.path.append('../')
from src.utils import get_data_abbr, mkdir_p, get_base_path, get_library_path
import pandas as pd
# from scipy.stats import kendalltau
import numpy as np

library_path = get_library_path()
sys.path.append(library_path)
sys.path.append(os.path.join(library_path, "hynetworkx"))

from src.data_preparer import filter_size, prepare_lp_data, get_time_filter_params,incidence_to_hyperedges, S_to_A
from src.hypergraph_link_predictor import get_hypergraph_scores, hypergraph_score_abbr_map, all_hypergraph_score_names
from src.link_predictor import get_perf_df
from src.linkpred_predictor import get_linkpred_scores, predictor_abbr_map, all_predictor_names
from src.supervised_link_predictor import classify
from src.experimenter import perform_GWH_classification
from src.incidence_matrix import parse_benson_incidence_matrix as parse_S
#from scr.tables_generator.ipynb import to_mean_std,get_overfit_score_df,get_data_split_name,get_latex_table,read_macro_feat_results,read_standalone_results
from experimenter import *
from utils import *
import pandas as pd
import numpy as np
from itertools import product
from tqdm import tqdm_notebook
from tabulate import tabulate
from joblib import Memory

base_path = get_base_path()

In [120]:
S1, times, id_label_map = parse_S(params['data_params']['data_name'],
                                     params['data_params']['base_path'],
                                     params['data_params']['split_mode'],
                                     params['data_params']['max_size_limit'],
                                     *get_time_filter_params(params['data_params']['data_name']))

1111111111111111111111111111111111111111111111
2222222222222222222222222222222222222222222222222
(8018, 8019)


In [167]:
base_path = '/content/gdrive/My Drive/Colab Notebooks/data/'
default_lp_indices=None
default_hyper_indices=None
data_params = {'data_name': 'contact-high-school',
               'base_path': base_path,
               'split_mode': 'structural',
               'max_size_limit': 10}
lp_data_params = {'rho': 0.2,
                  'neg_factor': 5,
                  'neg_mode': 'random',
                  'weighted_flag': False}
lp_params = {'linkpred_indices': None,  # Say None for all scores
             'hypergraph_score_indices': None,  # Say None for all scores
             }
classifier_params = {'features': None,
                     'classifier': 'xgboost'}
column_s = ['classifier']
data_names = ['women']
split_modes = ['structural']
weighted_flags = [False, True]

lp_cols = ['AA', 'AS', 'CN', 'Cos', 'PA', 'JC', 'MxO', 'MnO', 'NM', 'Prn']
metrics = ['ap', 'auc', 'p@+', 'p@10', 'p@100', 'r@+', 'r@10', 'r@100']
hyper_cols = ['HAAM', 'HAAa', 'HAAl1', 'HAAl2',
              'HASM', 'HASa', 'HASl1', 'HASl2',
              'HCNM', 'HCNa', 'HCNl1', 'HCNl2',
              'HCosM', 'HCosa', 'HCosl1', 'HCosl2',
              'HDPa', 'HPM', 'HPa', 'HPl1', 'HPl2',
              'HJCM', 'HJCa', 'HJCl1', 'HJCl2',
              'HmaxoM', 'HmaxoA', 'Hmaxol1', 'Hmaxol2',
              'HminoM', 'HminoA', 'Hminol1', 'Hminol2',
              'HNMM', 'HNMa', 'HNMl1', 'HNMl2',
              'HPearM', 'HPeara', 'HPearl1', 'HPearl2',
              ]

combined_tables = {}
mixed_combinations_map = {'AA': ['AA', 'HAAM', 'HAAa', 'HAAl1', 'HAAl2'],
                          'AS': ['AS', 'HASM', 'HASa', 'HASl1', 'HASl2'],
                          'CN': ['CN', 'HCNM', 'HCNa', 'HCNl1', 'HCNl2'],
                          'Cos': ['Cos', 'HCosM', 'HCosa', 'HCosl1', 'HCosl2'],
                          'PA': ['PA', 'HDPa', 'HPM', 'HPa', 'HPl1', 'HPl2'],
                          'JC': ['JC', 'HJCM', 'HJCa', 'HJCl1', 'HJCl2'],
                          'Kz': ['Kz', 'HKz'],
                          'MxO': ['MxO', 'HmaxoM', 'HmaxoA', 'Hmaxol1', 'Hmaxol2'],
                          'MnO': ['MnO', 'HminoM', 'HminoA', 'Hminol1', 'Hminol2'],
                          'NM': ['NM', 'HNMM', 'HNMa', 'HNMl1', 'HNMl2'],
                          'Prn': ['Prn', 'HPearM', 'HPeara', 'HPearl1', 'HPearl2'], }


data_params={'data_name': 'email-Enron',
                                      'base_path': base_path,
                                      'split_mode': 'temporal',
                                      'max_size_limit': 10}
lp_data_params={'rho': 0.2,
                                         'neg_factor': 5,
                                         'neg_mode': 'random'}
lp_params={'linkpred_indices': default_lp_indices,
                                    'hypergraph_score_indices': default_hyper_indices}
iter_var= 0

In [168]:
params = get_default_params()
params['data_params']['data_name'] = 'DAWN'
params['data_params']['split_mode'] = 'temporal'
params['data_params']['base_path'] = '/home2/e1-313-15477'
params['data_params']['max_size_limit'] =11
params['iter_var'] = 10000
a, lp_results ,S= perform_link_prediction(params['data_params'],
params['lp_data_params'],
params['lp_params'],
params['iter_var'])

In [169]:
c=lp_results['scores']
# c[['AA','w_AA', 'HAAM', 'HAAa', 'HAAl1', 'HAAl2']].rank(axis=1, ascending=False).T.sort_values('p@+', ascending=True).head(20)
#c[['PA', 'w_PA','HPM', 'HPa', 'HPl1', 'HPl2','label']].sort_values('PA', ascending=False).head(120).tail(60)


In [170]:
d=lp_results['perf']
# d[['AA','w_AA', 'HAAM', 'HAAa', 'HAAl1', 'HAAl2']].rank(axis=1, ascending=False).T.sort_values('p@+', ascending=True).head(20)
#d[['PA', 'w_PA','HPM', 'HPa', 'HPl1', 'HPl2']].T.sort_values('ap', ascending=False).head(10)

In [171]:
zz=lp_results['scores']

In [172]:
hes=incidence_to_hyperedges(S)

In [174]:
j=0
set_p_h=set()
for i in zz.index:
    q=frozenset(i)
    if q in hes:
        set_p_h.add(q)


In [175]:
scores=lp_results['scores']
scores_sub=scores[['HASs', 'HASl1', 'HASl2', 'HASl3', 'HNMm', 'HNMM', 'HNMa', 'HNMs',
       'HNMl1', 'HNMl2', 'HNMl3', 'HCosm', 'HCosM', 'HCosa', 'HCoss', 'HCosl1',
       'HCosl2', 'HCosl3', 'label']]
scores_sub.iloc[[10,33,29]]

HASs     HASl1     HASl2     HASl3  HNMm      HNMM      HNMa  \
0 117  3.209590  3.209590  0.520057  0.520057   0.0  0.392232  0.012684   
  595  4.899074  4.899074  0.710022  0.710022   0.0  0.392232  0.021038   
  525  0.000000  0.000000  0.000000  0.000000   0.0  0.000000  0.000000   

            HNMs      HNMl1     HNMl2     HNMl3  HCosm     HCosM     HCosa  \
0 117  11.365163  11.365163  1.696752  1.696752    0.0  0.408248  0.013556   
  595  15.989186  15.989186  2.091612  2.091612    0.0  0.408248  0.022247   
  525   0.000000   0.000000  0.000000  0.000000    0.0  0.000000  0.000000   

           HCoss     HCosl1    HCosl2    HCosl3  label  
0 117  12.146296  12.146296  1.810043  1.810043      1  
  595  16.907733  16.907733  2.213385  2.213385      1  
  525   0.000000   0.000000  0.000000  0.000000      1

In [176]:
pos=a['A_test_pos']
neg=a['A_test_neg']
xx=pos.todense()
yy=neg.todense()
xx[0,39] ==1 or xx[39,0]==1

True

In [177]:
size2=set()
for i in hes:
    if (len(i)==2):
        size2.add(i)

In [179]:
len(size2)

2549

In [178]:
ap=set()
for i in size2:
    for j in i:
        ap.add(j)
len(ap)

784

In [ ]:
df = lp_results['scores']

In [ ]:
df['is_original_edge'] = list(map(lambda x: x in size2, map(frozenset, list(df.index))))
df['is_original_edge'].value_counts()
original_edge_scores = df[((df['label'] == 1) & df['is_original_edge']) | (df['label'] == 0)]
original_edge_scores
cols = list(df.columns)
cols.remove('label')
cols.remove('is_original_edge')
perf_df = get_perf_df(original_edge_scores.drop('is_original_edge', axis=1), cols, [])

In [ ]:
approach = 'JC'
app1 = approach
app2 = 'H{}l2'.format(approach)

df1 = original_edge_scores.drop('is_original_edge', axis=1)
for c in cols:
#     df1[c] = (df1[c] - df1[c].min()) / (df1[c].max() - df1[c].min())
    df1[c] = df1[c].rank(ascending=False)
# df1['diff'] = df1[app1] - df1[app2]
# df1.sort_values('diff')[[a[, 'HAAl1', 'diff', 'label']]

top1 = sorted(df1[app1].unique())[:100]
top2 = sorted(df1[app2].unique())[:100]

bottom1 = [sorted(df1[app1].unique())[-2]]
bottom2 = [sorted(df1[app2].unique())[-2]]

PP1 = set(map(frozenset, df1[df1[app1].isin(top1)].index))
PP2 = set(map(frozenset, df1[df1[app2].isin(top2)].index))

PN1 = set(map(frozenset, df1[df1[app1].isin(bottom1)].index))
PN2 = set(map(frozenset, df1[df1[app2].isin(bottom2)].index))

AP = set(map(frozenset, df1[df1['label'] == 1].index))
AN = set(map(frozenset, df1[df1['label'] == 0].index))

TP1 = PP1.intersection(AP)
TP2 = PP2.intersection(AP)

FP1 = PP1.intersection(AN)
FP2 = PP2.intersection(AN)

FN1 = AP.difference(TP1)
FN2 = AP.difference(TP2)

list(map(len, [TP1, TP2, FP1, FP2, FN1, FN2]))

In [ ]:
#original_edge_scores.loc[map(lambda x: (min(x), max(x)), TP1.difference(TP2)), [app1, app2]]

In [ ]:
sorted(df1['AA'].unique())[-2]

In [ ]:
#df1['AA'].describe()

In [ ]:
#df1[['AA', 'HAAl1']].sum()

In [ ]:
c=lp_results['perf']
# c[['AA','w_AA', 'HAAM', 'HAAa', 'HAAl1', 'HAAl2']].rank(axis=1, ascending=False).T.sort_values('p@+', ascending=True).head(20)
c[['AA', 'w_AA','HAAM', 'HAAa', 'HAAl1', 'HAAl2']].T.sort_values('ap', ascending=False).head(10)

In [ ]:
c=lp_results['perf']
# c[['AA','w_AA', 'HAAM', 'HAAa', 'HAAl1', 'HAAl2']].rank(axis=1, ascending=False).T.sort_values('p@+', ascending=True).head(20)
c[['AS', 'w_AS','HASM', 'HASa', 'HASl1', 'HASl2']].T.sort_values('ap', ascending=False).head(10)

In [ ]:
c=lp_results['perf']
# c[['AA','w_AA', 'HAAM', 'HAAa', 'HAAl1', 'HAAl2']].rank(axis=1, ascending=False).T.sort_values('p@+', ascending=True).head(20)
c[['JC', 'w_JC','HJCM', 'HJCa', 'HJCl1', 'HJCl2']].T.sort_values('ap', ascending=False).head(10)

In [ ]:
qwe.columns

In [ ]:
qwe[['label']].sum()
qwe.drop('pred',axis=1)

In [ ]:
col = 'HCosl2'
qwe=qwe.sort_values(col, ascending=False)
top_scores = set(qwe[col].head(k)[col])
num_top_pairs = qwe[qwe[col].isin(top_scores)].shape[0]
qwe['pred_{}'.format(col)]=[1]*num_top_pairs + [0]*(qwe.shape[0]-num_top_pairs)

In [ ]:
# qwe[['pred_HAAl2', 'pred_HASl2' ,'pred_HJCl2', 'pred_Hmaxol2', 'pred_HCosl2', 'pred_HPearl2', 'pred_HCNl2', 'pred_Hminol2', 'pred_HPl2', 'pred_HNMl2','label']].sort_values('label',ascending=False).head(150).tail(30)
df = qwe[[c for c in qwe.columns if c.startswith('pred') or c == 'label']]
PP_map = {}
PN_map = {}
for c in df.columns:
        PP_map[c] = set(map(frozenset, df[df[c] == 1].index))
        PN_map[c] = set(map(frozenset, df[df[c] == 0].index))
# df[df['label'] == 1].sum()
TP_map = {c:PP_map[c].intersection(PP_map['label']) for c in PP_map}
FN_map = {c:PN_map[c].intersection(PP_map['label']) for c in PP_map}

In [ ]:
# TP_map['pred_AA']
from tqdm import tqdm_notebook
_dict = {}
hhh = set()
ggg = set()
for c1 in tqdm_notebook(TP_map):

    TP_1 = TP_map[c1]
    FN_1 = FN_map[c1]
    _dict[c1] = {}
    
    if (c1.endswith('1') or c1.endswith('2')):
#         print(c1)
        hhh=hhh.union(TP_1)
    else:
        ggg=ggg.union(TP_1)
        
    for c2 in TP_map:
        TP_2 = TP_map[c2]
        FN_2 = FN_map[c2]
        _dict[c1][c2] = len(TP_1.difference(TP_2))
#pd.DataFrame(_dict).iloc[22:, 2:12]

In [ ]:
len(hhh.difference(ggg))

In [ ]:
#pd.DataFrame(_dict).iloc[22:, 12:22]

In [ ]:
#qwer.loc[(103,704),:]
asd=[]
for i in default_lp_cols:
    asd.extend(mcm[i][1:])
asd

In [16]:
from experimenter import *
params = get_default_params()
params['data_params']['data_name'] = 'DAWN'
params['data_params']['split_mode'] = 'temporal'
params['data_params']['base_path'] = '/home2/e1-313-15477'
params['data_params']['max_size_limit'] =11
params['iter_var'] = 10000

from experimenter import *
from utils import *
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

# mcm = mixed_combinations_map
# if 'HDPa' in mcm['PA']:
#     mcm['PA'].remove('HDPa')
    
# G_feats = [default_lp_cols]
# W_feats = ['w_{}'.format(x) for x in default_lp_cols]
# H_feats = asd

G_feats=['AA', 'AS', 'CN', 'Cos', 'PA', 'JC', 'MxO', 'MnO', 'NM', 'Prn']
W_feats=['w_AA','w_AS', 'w_CN', 'w_Cos', 'w_PA', 'w_JC', 'w_MxO', 'w_MnO', 'w_NM','w_Prn']
H_feats=['HAAl1', 'HAAl2','HJCl1', 'HJCl2', 'Hmaxol1', 'Hmaxol2', 'HPearl1', 'HPearl2', 'HCNl1', 'HCNl2',
         'Hminol1','Hminol2', 'HPl1','HPl2','HASl1', 'HASl2','HNMl1', 'HNMl2',  'HCosl1','HCosl2']
output = perform_GWH_classification(params, G_feats, W_feats, H_feats, 'xgboost')


In [49]:
df = output[tuple(G_feats)]['test_scores'].rename(columns={'xgboost_test': 'G'})
df = df.merge(output[tuple(W_feats)]['test_scores'].rename(columns={'xgboost_test': 'W'}), left_index = True, right_index = True)
df = df.merge(output[tuple(H_feats)]['test_scores'].rename(columns={'xgboost_test': 'H'}), left_index = True, right_index = True)
df = df.merge(output[tuple(G_feats+H_feats)]['test_scores'].rename(columns={'xgboost_test': 'GH'}), left_index = True, right_index = True)
df = df.merge(output[tuple(W_feats+H_feats)]['test_scores'].rename(columns={'xgboost_test': 'WH'}), left_index = True, right_index = True)
pl_df = lp_results['scores'][['label']]
df = pl_df.merge(df, left_index = True, right_index = True, how='right')
df

,,label,G,W,H,GH,WH
6,1021,0,0.043656,0.043538,0.048023,0.048493,0.047789
397,1105,0,0.025188,0.039322,0.028418,0.029355,0.028922
544,710,0,0.043656,0.043538,0.048023,0.048493,0.047789
3,156,0,0.728203,0.577335,0.730160,0.715752,0.626289
536,701,0,0.043656,0.043538,0.011188,0.011385,0.013557
...,...,...,...,...,...,...,...
129,632,0,0.043656,0.043538,0.028418,0.029355,0.028922
397,1041,0,0.043656,0.043538,0.011188,0.011385,0.013557
69,855,0,0.947742,0.892804,0.832511,0.938593,0.897325
100,804,0,0.043656,0.043538,0.027225,0.029355,0.028922


In [58]:
qwe.copy(deep=True)
k = df['label'].value_counts()[1]

In [59]:
col = 'G'
for col in df.columns:
    if col == 'label':
        continue
    qwe=qwe.sort_values(col, ascending=False)
    top_scores = set(qwe.head(k)[col])
    num_top_pairs = qwe[qwe[col].isin(top_scores)].shape[0]
    qwe['pred_{}'.format(col)] = [1]*num_top_pairs + [0]*(qwe.shape[0]-num_top_pairs)
    

In [63]:
# qwe[['pred_HAAl2', 'pred_HASl2' ,'pred_HJCl2', 'pred_Hmaxol2', 'pred_HCosl2', 'pred_HPearl2', 'pred_HCNl2', 'pred_Hminol2', 'pred_HPl2', 'pred_HNMl2','label']].sort_values('label',ascending=False).head(150).tail(30)
df = qwe[[c for c in qwe.columns if c.startswith('pred') or c == 'label']]
PP_map = {}
PN_map = {}
for c in df.columns:
        PP_map[c] = set(map(frozenset, df[df[c] == 1].index))
        PN_map[c] = set(map(frozenset, df[df[c] == 0].index))
# df[df['label'] == 1].sum()
TP_map = {c:PP_map[c].intersection(PP_map['label']) for c in PP_map}
FN_map = {c:PN_map[c].intersection(PP_map['label']) for c in PP_map}

In [108]:
# TP_map['pred_AA']
from tqdm import tqdm_notebook
_dict = {}
hhh = set()
ggg = set()
for c1 in tqdm_notebook(TP_map):

    TP_1 = TP_map[c1]
    print(c1,len(TP_1))
    FN_1 = FN_map[c1]
    Prec_1 = len(TP_1)/k
    _dict[c1+'_1'] = {}
    
    if (c1.endswith('1') or c1.endswith('2')):
#         print(c1)
        hhh=hhh.union(TP_1)
    else:
        ggg=ggg.union(TP_1)
        
    for c2 in TP_map:
        

        TP_2 = TP_map[c2]
        FN_2 = FN_map[c2]
        print(c2,len(TP_2))
        Prec_2 = len(TP_2)/k
        _dict[c1+'_1'][c2+'_2'] = len(TP_1.difference(TP_2))
#pd.DataFrame(_dict).iloc[22:, 2:12]

label 1265
label 1265
pred_WH 875
pred_G 875
pred_W 878
pred_H 880
pred_GH 882
pred_WH 875
label 1265
pred_WH 875
pred_G 875
pred_W 878
pred_H 880
pred_GH 882
pred_G 875
label 1265
pred_WH 875
pred_G 875
pred_W 878
pred_H 880
pred_GH 882
pred_W 878
label 1265
pred_WH 875
pred_G 875
pred_W 878
pred_H 880
pred_GH 882
pred_H 880
label 1265
pred_WH 875
pred_G 875
pred_W 878
pred_H 880
pred_GH 882
pred_GH 882
label 1265
pred_WH 875
pred_G 875
pred_W 878
pred_H 880
pred_GH 882



In [97]:
len(TP_map['pred_G'].difference(TP_map['pred_GH']))

9

In [111]:
pd.DataFrame(_dict).T.iloc[[5, 1], [2, 3, 4]]

,pred_G_2,pred_W_2,pred_H_2
pred_GH_1,16,28,18
pred_WH_1,22,17,20


In [113]:
pd.DataFrame(_dict).T.iloc[[2, 3, 4], [5, 1]]

,pred_GH_2,pred_WH_2
pred_G_1,9,22
pred_W_1,24,20
pred_H_1,16,25


In [81]:
df = output[tuple(G_feats)]['test_perf'].rename(columns={'xgboost_test': 'G'})
df = df.merge(output[tuple(W_feats)]['test_perf'].rename(columns={'xgboost_test': 'W'}), left_index = True, right_index = True)
df = df.merge(output[tuple(H_feats)]['test_perf'].rename(columns={'xgboost_test': 'H'}), left_index = True, right_index = True)
df = df.merge(output[tuple(G_feats+H_feats)]['test_perf'].rename(columns={'xgboost_test': 'GH'}), left_index = True, right_index = True)
df = df.merge(output[tuple(W_feats+H_feats)]['test_perf'].rename(columns={'xgboost_test': 'WH'}), left_index = True, right_index = True)
# pl_df = lp_results['scores'][['label']]
# df = pl_df.merge(df, left_index = True, right_index = True, how='right')
df

,G,W,H,GH,WH
ap,71.49,72.00,71.79,72.41,73.02
auc,87.96,88.24,88.74,88.87,88.87
p@+,69.17,69.41,69.57,69.72,69.17
p@10,100.00,90.00,100.00,90.00,100.00
p@100,95.00,96.00,95.00,95.00,97.00
r@+,69.17,69.41,69.57,69.72,69.17
r@10,0.79,0.71,0.79,0.71,0.79
r@100,7.51,7.59,7.51,7.51,7.67


In [114]:
df = output[tuple(G_feats)]['train_perf'].rename(columns={'xgboost_train': 'G'})
df = df.merge(output[tuple(W_feats)]['train_perf'].rename(columns={'xgboost_train': 'W'}), left_index = True, right_index = True)
df = df.merge(output[tuple(H_feats)]['train_perf'].rename(columns={'xgboost_train': 'H'}), left_index = True, right_index = True)
df = df.merge(output[tuple(G_feats+H_feats)]['train_perf'].rename(columns={'xgboost_train': 'GH'}), left_index = True, right_index = True)
df = df.merge(output[tuple(W_feats+H_feats)]['train_perf'].rename(columns={'xgboost_train': 'WH'}), left_index = True, right_index = True)
# pl_df = lp_results['scores'][['label']]
# df = pl_df.merge(df, left_index = True, right_index = True, how='right')
df

,G,W,H,GH,WH
ap,74.30,74.80,75.12,75.94,76.69
auc,88.44,88.51,89.62,89.73,89.80
p@+,70.18,70.34,70.22,70.62,70.91
p@10,100.00,100.00,100.00,100.00,100.00
p@100,100.00,100.00,100.00,100.00,100.00
r@+,70.18,70.34,70.22,70.62,70.91
r@10,0.19,0.19,0.19,0.19,0.19
r@100,1.92,1.92,1.92,1.92,1.92


In [141]:
missed_pairs = TP_map['pred_G'].difference(TP_map['pred_GH'])

In [130]:
imp_pairs = TP_map['pred_GH'].difference(TP_map['pred_G'])

In [105]:
#len(TP_map['pred_G'].difference(TP_map['pred_H']))

In [151]:
df1 = output[tuple(G_feats)]['test_scores']

In [156]:
_list = []
for p in imp_pairs:
    u, v = list(p)
    a, b = id_label_map[u], id_label_map[v]
    S_a = S[u, :]
    S_b = S[v, :]
    print (' '.join(a.split()[1:]),
           S_a.sum(),
           '\t',
           ' '.join(b.split()[1:]),
           S_b.sum(),
           round(output[tuple(G_feats)]['test_scores'].loc[tuple(sorted(p)), 'xgboost_test'], 2),
           round(output[tuple(G_feats + H_feats)]['test_scores'].loc[tuple(sorted(p)), 'xgboost_test'], 2))
    _list.append(a)
    _list.append(b)
from collections import Counter
Counter(_list)

PANTOPRAZOLE 77 	 VANCOMYCIN 6 0.32 0.37
METHAMPHETAMINE 236 	 MESCALINE 2 0.34 0.42
BENZOCAINE TOPICAL 3 	 METHADONE 317 0.31 0.35
ATAZANAVIR 3 	 LAMIVUDINE 9 0.19 0.45
LOSARTAN 20 	 CLARITHROMYCIN 35 0.33 0.35
CARVEDILOL 25 	 HYDRALAZINE 11 0.28 0.36
RAMIPRIL 17 	 FOLIC ACID 24 0.29 0.39
PREDNISOLONE 7 	 HYDROXYZINE 131 0.31 0.36
MULTIVITAMIN WITH MINERALS 24 	 FOSINOPRIL 22 0.27 0.36
BUMETANIDE 5 	 SPIRONOLACTONE 36 0.28 0.37
TENOFOVIR 13 	 EMTRICITABINE 3 0.1 0.5
BECLOMETHASONE 3 	 HYDROXYZINE 131 0.31 0.36
VALSARTAN 36 	 OXYBUTYNIN 13 0.31 0.36
AZITHROMYCIN 62 	 NYSTATIN 10 0.3 0.41
FLUOXETINE 200 	 INTERFERONS-NOS 7 0.34 0.35
PYRAZINAMIDE 3 	 LEVOFLOXACIN 57 0.31 0.4


Counter({'D04514 PANTOPRAZOLE': 1,
         'D00125 VANCOMYCIN': 1,
         'D00805 METHAMPHETAMINE': 1,
         'J00178 MESCALINE': 1,
         'D00813 BENZOCAINE TOPICAL': 1,
         'D00050 METHADONE': 1,
         'D04882 ATAZANAVIR': 1,
         'D03858 LAMIVUDINE': 1,
         'D03821 LOSARTAN': 1,
         'D00097 CLARITHROMYCIN': 1,
         'D03847 CARVEDILOL': 1,
         'D00132 HYDRALAZINE': 1,
         'D00728 RAMIPRIL': 1,
         'D00241 FOLIC ACID': 1,
         'D00084 PREDNISOLONE': 1,
         'D00907 HYDROXYZINE': 2,
         'D03145 MULTIVITAMIN WITH MINERALS': 1,
         'D00242 FOSINOPRIL': 1,
         'D00179 BUMETANIDE': 1,
         'D00373 SPIRONOLACTONE': 1,
         'D04774 TENOFOVIR': 1,
         'D04884 EMTRICITABINE': 1,
         'D00760 BECLOMETHASONE': 1,
         'D04113 VALSARTAN': 1,
         'D00328 OXYBUTYNIN': 1,
         'D00091 AZITHROMYCIN': 1,
         'D01233 NYSTATIN': 1,
         'D00236 FLUOXETINE': 1,
         'J01243 INTERFERONS-NOS':

In [162]:
for p in imp_pairs:
    u, v = list(p)
    a, b = id_label_map[u], id_label_map[v]
    S_a = S[u, :]
    S_b = S[v, :]
    print(S_a.sum())
    print(S_b.sum())

77
6
236
2
3
317
3
9
20
35
25
11
17
24
7
131
24
22
5
36
13
3
3
131
36
13
62
10
200
7
3
57


In [163]:
df = lp_results['scores']

In [164]:
df

AA        AS   CN       Cos      PA        JC       MxO  \
0    1     0.000000  0.000000  0.0  0.000000   135.0  0.000000  0.000000   
     9     0.811912  0.006504  4.0  0.161296   615.0  0.076923  0.097561   
     15    0.000000  0.000000  0.0  0.000000     0.0  0.000000  0.000000   
     27    0.000000  0.000000  0.0  0.000000   105.0  0.000000  0.000000   
     37    1.689131  0.005993  8.0  0.218952  1335.0  0.083333  0.089888   
...             ...       ...  ...       ...     ...       ...       ...   
1186 1192  0.000000  0.000000  0.0  0.000000     0.0  0.000000  0.000000   
1188 1190  0.000000  0.000000  0.0  0.000000     0.0  0.000000  0.000000   
     1192  0.000000  0.000000  0.0  0.000000     0.0  0.000000  0.000000   
1190 1194  0.000000  0.000000  0.0  0.000000     0.0  0.000000  0.000000   
1192 1193  0.000000  0.000000  0.0  0.000000     0.0  0.000000  0.000000   

                MnO        NM       Prn  ...     HNMl2     HNMl3  HCosm  \
0    1     0.000000  0.000000  0.000000  ...  0.000000  0.000000    0.0   
     9     0.266667  0.129573  0.143954  ...  1.183543  1.183543    0.0   
     15    0.000000  0.000000  0.000000  ...  0.000000  0.000000    0.0   
     27    0.000000  0.000000  0.000000  ...  0.000000  0.000000    0.0   
     37    0.533333  0.125352  0.197088  ...  1.151574  1.151574    0.0   
...             ...       ...       ...  ...       ...       ...    ...   
1186 1192  0.000000  0.000000  0.000000  ...  0.000000  0.000000    0.0   
1188 1190  0.000000  0.000000  0.000000  ...  0.000000  0.000000    0.0   
     1192  0.000000  0.000000  0.000000  ...  0.000000  0.000000    0.0   
1190 1194  0.000000  0.000000  0.000000  ...  0.000000  0.000000    0.0   
1192 1193  0.000000  0.000000  0.000000  ...  0.000000  0.000000    0.0   

              HCosM     HCosa     HCoss    HCosl1    HCosl2    HCosl3  label  
0    1     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000      0  
     9     0.500000  0.017170  4.944949  4.944949  1.227375  1.227375      0  
     15    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000      0  
     27    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000      0  
     37    0.447214  0.012254  5.783834  5.783834  1.235062  1.235062      0  
...             ...       ...       ...       ...       ...       ...    ...  
1186 1192  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000      0  
1188 1190  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000      0  
     1192  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000      0  
1190 1194  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000      0  
1192 1193  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000      0  

[38820 rows x 92 columns]